In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign


Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0


In [ ]:
import zipfile

with zipfile.ZipFile("gtsrb-german-traffic-sign.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


In [ ]:
import os
import pandas as pd

# List files
os.listdir('/content/dataset')

# Check CSV structure
df = pd.read_csv('/content/dataset/Train.csv')
df.head()


,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [ ]:
import shutil
import os

# Recreate the directories
os.makedirs('/content/yolo_dataset/images/train', exist_ok=True)
os.makedirs('/content/yolo_dataset/labels/train', exist_ok=True)



In [ ]:



## ✅ Fix: Correct Script to Flatten Images + Labels to `train/`


from tqdm import tqdm
import cv2
import shutil
import os
import pandas as pd

df = pd.read_csv('/content/dataset/Train.csv')

# Make sure the folders exist
os.makedirs('/content/yolo_dataset/images/train', exist_ok=True)
os.makedirs('/content/yolo_dataset/labels/train', exist_ok=True)

for index, row in tqdm(df.iterrows(), total=len(df)):
    filename = row['Path']  # e.g., Train/42/00042_00007_00029.png
    class_id = row['ClassId']

    img_path = os.path.join('/content/dataset/', filename)

    if not os.path.exists(img_path):
        print(f"Missing: {img_path}")
        continue

    img = cv2.imread(img_path)
    if img is None:
        print(f"Can't read: {img_path}")
        continue

    h, w, _ = img.shape

    # YOLO format conversion
    x1, y1, x2, y2 = row['Roi.X1'], row['Roi.Y1'], row['Roi.X2'], row['Roi.Y2']
    x_center = ((x1 + x2) / 2) / w
    y_center = ((y1 + y2) / 2) / h
    bbox_width = (x2 - x1) / w
    bbox_height = (y2 - y1) / h

    # Save to flat path
    image_filename = filename.split('/')[-1]  # e.g., 00042_00007_00029.png

    dest_img_path = f"/content/yolo_dataset/images/train/{image_filename}"
    shutil.copy(img_path, dest_img_path)

    label_filename = image_filename.replace(".ppm", ".txt").replace(".png", ".txt")
    with open(f"/content/yolo_dataset/labels/train/{label_filename}", "w") as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")


100%|██████████| 39209/39209 [00:28<00:00, 1359.37it/s]


In [ ]:
class_names = [
    "Speed limit (20km/h)", "Speed limit (30km/h)", "Speed limit (50km/h)",
    "Speed limit (60km/h)", "Speed limit (70km/h)", "Speed limit (80km/h)",
    "End of speed limit (80km/h)", "Speed limit (100km/h)", "Speed limit (120km/h)",
    "No passing", "No passing for vehicles over 3.5 metric tons", "Right-of-way at the next intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5 metric tons prohibited",
    "No entry", "General caution", "Dangerous curve to the left", "Dangerous curve to the right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right", "Road work",
    "Traffic signals", "Pedestrians", "Children crossing", "Bicycles crossing", "Beware of ice/snow",
    "Wild animals crossing", "End of all speed and passing limits", "Turn right ahead",
    "Turn left ahead", "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout mandatory", "End of no passing", "End of no passing by vehicles over 3.5 metric tons"
]



In [ ]:
yaml_content = f"""
train: /content/yolo_dataset/images/train
val: /content/yolo_dataset/images/train  # reuse train for now, or split later

nc: {len(class_names)}
names: {class_names}
"""

with open("/content/yolo_dataset/data.yaml", "w") as f:
    f.write(yaml_content)



In [ ]:
# Clone YOLOv5 from Ultralytics GitHub
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install required Python packages
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17326 (from 3)
Receiving objects: 100% (17410/17410), 16.31 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (11931/11931), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s e

In [ ]:
%%shell
python train.py \
  --img 416 \
  --batch 16 \
  --epochs 30 \
  --data /content/yolo_dataset/data.yaml \
  --weights yolov5s.pt \
  --name traffic_signs_yolov5



Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      15/29       2.3G    0.01501   0.007445    0.02738         41        416:  65% 1588/2451 [04:43<02:15,  6.36it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      15/29       2.3G    0.01501   0.007446    0.02739         45        416:  65% 1589/2451 [04:43<02:13,  6.48it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      15/29       2.3G    0.01501   0.007445    0.02738         33        416:  65% 1590/2451 [04:43<02:19,  6.16it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.aut

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/train/traffic_signs_yolov5/weights/best.pt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell
python train.py \
  --img 416 \
  --batch 16 \
  --epochs 5 \
  --data /content/yolo_dataset/data.yaml \
  --weights yolov5s.pt \
  --name traffic_signs_yolov5 \
  --project /content/drive/MyDrive/yolo_training

In [ ]:
%%shell
python train.py \
  --img 416 \
  --batch 16 \
  --epochs 20 \
  --data /content/yolo_dataset/data.yaml \
  --weights /content/drive/MyDrive/yolo_training/traffic_signs_yolov5/weights/last.pt \
  --name traffic_signs_yolov5 \
  --project /content/drive/MyDrive/yolo_training


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-28 19:44:54.323006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745869494.362671    2871 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745869494.376332    2871 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
wandb: WARNING If you're specifying your api key in code, ensure this code is 

CalledProcessError: Command 'python train.py \
  --img 416 \
  --batch 16 \
  --epochs 20 \
  --data /content/yolo_dataset/data.yaml \
  --weights /content/drive/MyDrive/yolo_training/traffic_signs_yolov5/weights/last.pt \
  --name traffic_signs_yolov5 \
  --project /content/drive/MyDrive/yolo_training
' returned non-zero exit status 1.